# OpenAI埋め込みのベクターデータベースとしてPolarDB-PGを使用する

このノートブックでは、OpenAI埋め込みのベクターデータベースとしてPolarDB-PGを使用する方法を段階的にガイドします。

このノートブックでは、以下のエンドツーエンドのプロセスを紹介します：
1. OpenAI APIによって作成された事前計算済み埋め込みの使用
2. PolarDB-PGのクラウドインスタンスへの埋め込みの保存
3. OpenAI APIを使用した生テキストクエリの埋め込みへの変換
4. PolarDB-PGを使用した作成されたコレクションでの最近傍検索の実行

### PolarDB-PGとは

[PolarDB-PG](https://www.alibabacloud.com/help/en/polardb/latest/what-is-polardb-2)は、読み書き分離アーキテクチャを採用した高性能ベクターデータベースです。Alibaba Cloudによって管理されるクラウドネイティブデータベースで、PostgreSQLと100%互換性があり、Oracle構文との高い互換性を持っています。大規模なベクターデータの保存とクエリの処理をサポートし、基盤となる実行アルゴリズムの最適化によってベクター計算の効率を大幅に向上させ、ユーザーに高速、弾力的、高性能、大容量ストレージ、安全で信頼性の高いベクターデータベースサービスを提供します。さらに、PolarDB-PGは多次元・マルチモーダル時空間情報エンジンと地理情報エンジンもサポートしています。同時に、PolarDB-PGは完全なOLAP機能とサービスレベル契約を備えており、多くのユーザーに認められ使用されています。

### デプロイメントオプション

- [PolarDB-PGクラウドベクターデータベース](https://www.alibabacloud.com/product/polardb-for-postgresql)を使用。[こちらをクリック](https://www.alibabacloud.com/product/polardb-for-postgresql?spm=a3c0i.147400.6791778070.243.9f204881g5cjpP)して高速デプロイメント。

## 前提条件

この演習を行うために、いくつかの準備が必要です：

1. PolarDB-PGクラウドサーバーインスタンス
2. ベクトルデータベースとやり取りするための'psycopg2'ライブラリ。他のPostgreSQLクライアントライブラリでも構いません。
3. [OpenAI APIキー](https://beta.openai.com/account/api-keys)

サーバーが正常に起動したかどうかを確認するために、簡単な curl コマンドを実行して検証することができます：

### 必要なパッケージのインストール

このノートブックは明らかに`openai`と`psycopg2`パッケージが必要ですが、使用する他の追加ライブラリもいくつかあります。以下のコマンドですべてをインストールできます：

In [ ]:
! pip install openai psycopg2 pandas wget

OpenAI APIキーを準備する
OpenAI APIキーは、ドキュメントとクエリのベクトル化に使用されます。

OpenAI APIキーをお持ちでない場合は、https://beta.openai.com/account/api-keys から取得できます。

キーを取得したら、環境変数として`OPENAI_API_KEY`に追加してください。

環境変数を通じてAPIキーを設定することについて疑問がある場合は、[APIキー安全性のベストプラクティス](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)を参照してください。

In [3]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.

if os.getenv("OPENAI_API_KEY") is not None:
    print("OPENAI_API_KEY is ready")
else:
    print("OPENAI_API_KEY environment variable not found")

OPENAI_API_KEY is ready


## PolarDBに接続する
まず環境変数に追加してください。または、以下の"psycopg2.connect"パラメータを直接変更することもできます。

実行中のPolarDBサーバーインスタンスへの接続は、公式のPythonライブラリを使用すると簡単です：

In [4]:
import os
import psycopg2

# Note. alternatively you can set a temporary env variable like this:
# os.environ["PGHOST"] = "your_host"
# os.environ["PGPORT"] "5432"),
# os.environ["PGDATABASE"] "postgres"),
# os.environ["PGUSER"] "user"),
# os.environ["PGPASSWORD"] "password"),

connection = psycopg2.connect(
    host=os.environ.get("PGHOST", "localhost"),
    port=os.environ.get("PGPORT", "5432"),
    database=os.environ.get("PGDATABASE", "postgres"),
    user=os.environ.get("PGUSER", "user"),
    password=os.environ.get("PGPASSWORD", "password")
)

# Create a new cursor object
cursor = connection.cursor()

利用可能な任意のメソッドを実行して接続をテストできます：

In [5]:
# Execute a simple query to test the connection
cursor.execute("SELECT 1;")
result = cursor.fetchone()

# Check the query result
if result == (1,):
    print("Connection successful!")
else:
    print("Connection failed.")

Connection successful!


In [7]:
import wget

embeddings_url = "https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip"

# The file is ~700 MB so this will take some time
wget.download(embeddings_url)

'vector_database_wikipedia_articles_embedded.zip'

ダウンロードしたファイルを展開する必要があります：

In [8]:
import zipfile
import os
import re
import tempfile

current_directory = os.getcwd()
zip_file_path = os.path.join(current_directory, "vector_database_wikipedia_articles_embedded.zip")
output_directory = os.path.join(current_directory, "../../data")

with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(output_directory)


# check the csv file exist
file_name = "vector_database_wikipedia_articles_embedded.csv"
data_directory = os.path.join(current_directory, "../../data")
file_path = os.path.join(data_directory, file_name)


if os.path.exists(file_path):
    print(f"The file {file_name} exists in the data directory.")
else:
    print(f"The file {file_name} does not exist in the data directory.")

The file vector_database_wikipedia_articles_embedded.csv exists in the data directory.


## インデックスデータ

PolarDBは__relation__にデータを格納し、各オブジェクトは少なくとも1つのベクトルで記述されます。私たちのrelationは**articles**と呼ばれ、各オブジェクトは**title**と**content**の両方のベクトルで記述されます。

まず、relationを作成し、**title**と**content**の両方にベクトルインデックスを作成してから、事前に計算された埋め込みでそれを埋めていきます。

In [6]:
create_table_sql = '''
CREATE TABLE IF NOT EXISTS public.articles (
    id INTEGER NOT NULL,
    url TEXT,
    title TEXT,
    content TEXT,
    title_vector vector(1536),
    content_vector vector(1536),
    vector_id INTEGER
);

ALTER TABLE public.articles ADD PRIMARY KEY (id);
'''

# SQL statement for creating indexes
create_indexes_sql = '''
CREATE INDEX ON public.articles USING ivfflat (content_vector) WITH (lists = 1000);

CREATE INDEX ON public.articles USING ivfflat (title_vector) WITH (lists = 1000);
'''

# Execute the SQL statements
cursor.execute(create_table_sql)
cursor.execute(create_indexes_sql)

# Commit the changes
connection.commit()

## データの読み込み

このセクションでは、このセッションの前に準備されたデータを読み込みます。これにより、あなた自身のクレジットを使ってWikipedia記事の埋め込みを再計算する必要がありません。

In [ ]:
import io

# Path to your local CSV file
csv_file_path = '../../data/vector_database_wikipedia_articles_embedded.csv'

# Define a generator function to process the file line by line
def process_file(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line

# Create a StringIO object to store the modified lines
modified_lines = io.StringIO(''.join(list(process_file(csv_file_path))))

# Create the COPY command for the copy_expert method
copy_command = '''
COPY public.articles (id, url, title, content, title_vector, content_vector, vector_id)
FROM STDIN WITH (FORMAT CSV, HEADER true, DELIMITER ',');
'''

# Execute the COPY command using the copy_expert method
cursor.copy_expert(copy_command, modified_lines)

# Commit the changes
connection.commit()

In [9]:
# Check the collection size to make sure all the points have been stored
count_sql = """select count(*) from public.articles;"""
cursor.execute(count_sql)
result = cursor.fetchone()
print(f"Count:{result[0]}")

Count:25000


## データの検索

データがQdrantに格納されたら、最も近いベクトルを求めてコレクションにクエリを開始します。タイトルベースの検索からコンテンツベースの検索に切り替えるために、追加パラメータ`vector_name`を提供することができます。事前計算された埋め込みは`text-embedding-3-small` OpenAIモデルで作成されたため、検索時にも同じモデルを使用する必要があります。

In [10]:
def query_polardb(query, collection_name, vector_name="title_vector", top_k=20):

    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(
        input=query,
        model="text-embedding-3-small",
    )["data"][0]["embedding"]

    # Convert the embedded_query to PostgreSQL compatible format
    embedded_query_pg = "[" + ",".join(map(str, embedded_query)) + "]"

    # Create SQL query
    query_sql = f"""
    SELECT id, url, title, l2_distance({vector_name},'{embedded_query_pg}'::VECTOR(1536)) AS similarity
    FROM {collection_name}
    ORDER BY {vector_name} <-> '{embedded_query_pg}'::VECTOR(1536)
    LIMIT {top_k};
    """
    # Execute the query
    cursor.execute(query_sql)
    results = cursor.fetchall()

    return results

In [11]:
import openai

query_results = query_polardb("modern art in Europe", "Articles")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. Museum of Modern Art (Score: 0.5)
2. Western Europe (Score: 0.485)
3. Renaissance art (Score: 0.479)
4. Pop art (Score: 0.472)
5. Northern Europe (Score: 0.461)
6. Hellenistic art (Score: 0.457)
7. Modernist literature (Score: 0.447)
8. Art film (Score: 0.44)
9. Central Europe (Score: 0.439)
10. European (Score: 0.437)
11. Art (Score: 0.437)
12. Byzantine art (Score: 0.436)
13. Postmodernism (Score: 0.434)
14. Eastern Europe (Score: 0.433)
15. Europe (Score: 0.433)
16. Cubism (Score: 0.432)
17. Impressionism (Score: 0.432)
18. Bauhaus (Score: 0.431)
19. Surrealism (Score: 0.429)
20. Expressionism (Score: 0.429)


In [12]:
# This time we'll query using content vector
query_results = query_polardb("Famous battles in Scottish history", "Articles", "content_vector")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. Battle of Bannockburn (Score: 0.489)
2. Wars of Scottish Independence (Score: 0.474)
3. 1651 (Score: 0.457)
4. First War of Scottish Independence (Score: 0.452)
5. Robert I of Scotland (Score: 0.445)
6. 841 (Score: 0.441)
7. 1716 (Score: 0.441)
8. 1314 (Score: 0.429)
9. 1263 (Score: 0.428)
10. William Wallace (Score: 0.426)
11. Stirling (Score: 0.419)
12. 1306 (Score: 0.419)
13. 1746 (Score: 0.418)
14. 1040s (Score: 0.414)
15. 1106 (Score: 0.412)
16. 1304 (Score: 0.411)
17. David II of Scotland (Score: 0.408)
18. Braveheart (Score: 0.407)
19. 1124 (Score: 0.406)
20. July 27 (Score: 0.405)
